In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import requests
from bs4 import BeautifulSoup

In [3]:
API_KEY = 'xxx'
BASE_URL = 'https://api.meetup.com'

event_id = '224498084'
urlname = 'DS-ProD-SF'

In [2]:
# prepare test data
df = pd.read_json('data2.json')
df_events_validate = df[df['event_id'] != 224498084]
# df_events_validate
df_rsvps_validate = df.groupby('member_id', axis = 0).sum()
# df_rsvps_validate
df_validate = pd.merge(df_events_validate, df_rsvps_validate, on = 'event_id')
# df_validate
df_validate.drop('event_id', axis = 1, inplace = True)
# df_validate
df_validate_2 = df_validate.groupby('member_id', axis = 0).sum()
df_validate_2 


,event_description
member_id,
292159,Get advice for ramping up your Data Science sk...
327482,"No it isn't Pair Programming.. It's ""Pair Data..."
493376,Get advice for ramping up your Data Science sk...
2853561,Get advice for ramping up your Data Science sk...
2964467,Get advice for ramping up your Data Science sk...
2966361,You've heard of Pair Programming: Introducing ...
3456622,Get advice for ramping up your Data Science sk...
3756931,Get advice for ramping up your Data Science sk...
3881071,You've heard of Pair Programming: Introducing ...


In [21]:
event_url = BASE_URL + '/' + urlname + '/events' +  '/' + event_id
params = {'sign':'true','key': API_KEY}
response = requests.get(event_url, params = params)
event = response.json()


event_desc_raw= event['description']

# event_desc_raw

soup = BeautifulSoup(event_desc_raw, 'html.parser')

upcomings = [soup.get_text()]

member_id = df_validate_2.index

In [44]:
# predicted rsvps(write a class of the model, use the model directly 
# on prepared testing data)

vectorizer = TfidfVectorizer(stop_words='english')
vectors = vectorizer.fit_transform(df_validate_2['event_description']).toarray()
# words = vectorizer.get_feature_names()
# print words

tokenized_upcomings = vectorizer.transform(upcomings)
cosine_similarities = linear_kernel(tokenized_upcomings, vectors)

# print cosine_similarities

def get_top_values(lst, n, labels):
    '''
    INPUT: LIST, INTEGER, LIST
    OUTPUT: LIST

    Given a list of values, find the indices with the highest n values.
    Return the labels for each of these indices.

    e.g.
    lst = [7, 3, 2, 4, 1]
    n = 2
    labels = ["cat", "dog", "mouse", "pig", "rabbit"]
    output: ["cat", "pig"]
    '''
    return [labels[i] for i in np.argsort(lst)[-1:-n-1:-1]]

for i, upcoming in enumerate(upcomings):
#     print upcoming
    print get_top_values(cosine_similarities[i], 132, member_id)
    print


[14640829, 36208302, 131968622, 107678252, 12018983, 3881071, 124748422, 132117532, 131106802, 127987752, 193048983, 123286652, 136614482, 116534582, 115799012, 104272062, 102406202, 132130532, 193049939, 138818732, 91290012, 144044892, 149624662, 150797922, 159490792, 162260602, 173000122, 176226512, 182667296, 183352972, 184168379, 184519851, 93918602, 84156452, 89894632, 47391462, 7419036, 9189781, 9789480, 10978618, 12632109, 13372294, 13571373, 13629527, 14486078, 14522782, 22928311, 44789882, 50412492, 84732272, 50558562, 51312542, 52853902, 58914472, 59595312, 63445652, 63486122, 64039142, 66798272, 67074242, 76592822, 184787736, 184556461, 117629542, 187899709, 192721871, 192773702, 192315146, 185967268, 192752466, 189868858, 189758798, 186425629, 190151606, 192127676, 192495024, 192182636, 186070708, 192454662, 188660793, 189962112, 189924552, 191020372, 192929104, 191119754, 190511118, 190752194, 189271820, 193000723, 187050989, 8645227, 12580731, 101970742, 4252347, 87429312

In [47]:
# actual rsvps - api
# get rsvps


member_id_actual = []
member_name = []


rsvps_url = BASE_URL + '/' + urlname +'/events/' + event_id + '/rsvps'
params = {'sign':'true','key': API_KEY}
response = requests.get(rsvps_url, params = params)
rsvps = response.json()

for rsvp in rsvps:
    member_id_actual.append(rsvp['member']['id'])
    member_name.append(rsvp['member']['name'])

member_id_actual.sort()
member_id_actual
# len(member_id_actual)

[2811577,
 2853561,
 2877764,
 2964467,
 3456622,
 3893698,
 4252347,
 4450970,
 4529261,
 4535468,
 4606034,
 5459364,
 6945361,
 7470257,
 7558057,
 8155925,
 8645227,
 9597041,
 9695541,
 10261008,
 10313922,
 11103435,
 11194418,
 11385042,
 11607094,
 11758447,
 11947024,
 11979817,
 12114353,
 12364192,
 12376838,
 12632109,
 13046928,
 13217331,
 13361447,
 13423381,
 13724645,
 14101550,
 14223503,
 14250450,
 14517158,
 14547520,
 14640829,
 15614191,
 19650431,
 25845992,
 27594592,
 27729702,
 28347392,
 29411742,
 30445632,
 32639182,
 34943932,
 36208302,
 47229142,
 52218022,
 58914472,
 62727302,
 63445652,
 65168872,
 67814862,
 73488052,
 79517952,
 82110232,
 87429312,
 89815292,
 92949702,
 97632302,
 98083842,
 98566432,
 102819752,
 102899012,
 104272062,
 104279872,
 107678252,
 110561312,
 111484472,
 113464982,
 115673952,
 116534582,
 117597952,
 117621092,
 119216172,
 123021512,
 123286652,
 132372822,
 134342202,
 135771772,
 136830692,
 138068862,
 14307023

In [41]:
predicted = [14640829, 36208302, 131968622, 107678252, 12018983, 3881071, 124748422, 132117532, 131106802, 127987752, 193048983, 123286652, 136614482, 116534582, 115799012, 104272062, 102406202, 132130532, 193049939, 138818732, 91290012, 144044892, 149624662, 150797922, 159490792, 162260602, 173000122, 176226512, 182667296, 183352972, 184168379, 184519851, 93918602, 84156452, 89894632, 47391462, 7419036, 9189781, 9789480, 10978618, 12632109, 13372294, 13571373, 13629527, 14486078, 14522782, 22928311, 44789882, 50412492, 84732272, 50558562, 51312542, 52853902, 58914472, 59595312, 63445652, 63486122, 64039142, 66798272, 67074242, 76592822, 184787736, 184556461, 117629542, 187899709, 192721871, 192773702, 192315146, 185967268, 192752466, 189868858, 189758798, 186425629, 190151606, 192127676, 192495024, 192182636, 186070708, 192454662, 188660793, 189962112, 189924552, 191020372, 192929104, 191119754, 190511118, 190752194, 189271820, 193000723, 187050989, 8645227, 12580731, 101970742, 4252347, 87429312, 82110232, 187202519, 186618958, 191199018, 25845992, 56812812, 187664188, 13423381, 191138424, 119406272, 151154102, 17451801, 151768462, 96063072, 13361447, 165510312, 181883622, 98566432, 102899012, 4529261, 10261008, 3756931, 89815292, 2964467, 2853561, 184957854, 104414772, 96744352, 11103435, 6276332, 130491272, 125514252, 12334140, 87574642, 113152052, 153558732, 141171262]

In [45]:
counter = 0
for i in predicted:
    if i in member_id_actual:
        counter += 1
print counter    
print counter/132.0

8
0.0606060606061
